In [1]:
import dask.dataframe as dd
import numpy as np
import os
import shutil

def create_non_iid_partitions(input_dir, num_clients, alpha):

    print("Đang đọc dữ liệu Parquet từ thư mục...")
    df = dd.read_parquet(input_dir)
    

    df = df.persist() 
    n_samples = len(df)
    print(f"Tổng số mẫu trong dữ liệu: {n_samples}")

    labels = sorted(df['label'].unique().compute())
    num_labels = len(labels)
    print(f"Tìm thấy {num_labels} nhãn duy nhất: {labels}")

    label_distribution = np.random.dirichlet([alpha] * num_labels, num_clients)
    
    client_dataframes = [[] for _ in range(num_clients)]

    for label_id in labels:
        print(f"  - Đang xử lý nhãn: {label_id}")
        
        label_df = df[df['label'] == label_id]
        label_df = label_df.sample(frac=1, random_state=42)


        proportions_for_label = label_distribution[:, label_id]
        

        fractions = list(proportions_for_label / proportions_for_label.sum())
        
        if len(label_df) > 0:
            split_dfs = label_df.random_split(fractions, random_state=42)
            
            # Gán mỗi phần đã chia vào client tương ứng
            for client_id in range(num_clients):
                client_dataframes[client_id].append(split_dfs[client_id])

    print("\nĐang tạo và lưu các phân vùng Non-IID...")
    for client_id in range(num_clients):
        output_path = f"FL_nonIID/file{client_id}"
        
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        
        if client_dataframes[client_id]:
            final_client_df = dd.concat(client_dataframes[client_id], axis=0)
            
            # Lưu ra thư mục Parquet riêng
            final_client_df.to_parquet(output_path, write_index=False, engine='pyarrow')
            print(f"Đã lưu thành công dữ liệu cho Client {client_id} vào thư mục: {output_path}")
        else:
            print(f"Client {client_id} không có dữ liệu nào được gán.")

    return [f"FL_nonIID/file{i}" for i in range(num_clients)]


if __name__ == '__main__':
    # --- CẤU HÌNH ---
    input_dir = "scaled_output_parquet" 
    num_clients = 3
    alpha = 0.5 

    client_dirs = create_non_iid_partitions(input_dir, num_clients, alpha)
    
    print("\n--- Phân phối nhãn trên các tập dữ liệu Non-IID đã tạo ---")
    for i, client_dir in enumerate(client_dirs):
        print(f"\nClient {i} (từ thư mục: {client_dir}):")
        client_df_check = dd.read_parquet(client_dir)
        label_counts = client_df_check['label'].value_counts().compute()
        print(label_counts)

Đang đọc dữ liệu Parquet từ thư mục...
Tổng số mẫu trong dữ liệu: 8536019
Tìm thấy 5 nhãn duy nhất: [0, 1, 2, 3, 4]
  - Đang xử lý nhãn: 0
  - Đang xử lý nhãn: 1
  - Đang xử lý nhãn: 2
  - Đang xử lý nhãn: 3
  - Đang xử lý nhãn: 4

Đang tạo và lưu các phân vùng Non-IID...
Đã lưu thành công dữ liệu cho Client 0 vào thư mục: FL_nonIID/file0
Đã lưu thành công dữ liệu cho Client 1 vào thư mục: FL_nonIID/file1
Đã lưu thành công dữ liệu cho Client 2 vào thư mục: FL_nonIID/file2

--- Phân phối nhãn trên các tập dữ liệu Non-IID đã tạo ---

Client 0 (từ thư mục: FL_nonIID/file0):
label
2    587873
0     99361
1    120669
4    283959
3     10320
Name: count, dtype: int64

Client 1 (từ thư mục: FL_nonIID/file1):
label
2    109366
0    922722
1    854888
4      1689
3    201097
Name: count, dtype: int64

Client 2 (từ thư mục: FL_nonIID/file2):
label
2    1936885
0      76112
1    3024456
4      31535
3     275087
Name: count, dtype: int64
